In [4]:
CREATE_CSV_FILES = True

In [5]:
import numpy as np
import librosa
import csv
import os

def extractWavFeatures(soundFilesFolder,indx):
    
    for filename in os.listdir(soundFilesFolder):
        y, sr = librosa.load(soundFilesFolder+'/'+filename, mono=True)
        y, index = librosa.effects.trim(y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=40)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {indx}'
        writer.writerow(to_append.split())

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 41):
    header += f' mfcc{i}'
header += ' number'
header = header.split()

file = open("train.txt", 'w')
writer = csv.writer(file)
writer.writerow(header)



if (CREATE_CSV_FILES == True):
    extractWavFeatures("data/Abdelrahman/open", 1)
    extractWavFeatures("data/Abdelrahman/other", 2)
    extractWavFeatures("data/Nasser/open",3)
    extractWavFeatures("data/Nasser/other",4)
    #extractWavFeatures("data/Esraa", 2)
    #extractWavFeatures("data/Mariam", 4)
    #extractWavFeatures("data/others", 0)

file.close()

In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

data = pd.read_csv("train.txt")

data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc32,mfcc33,mfcc34,mfcc35,mfcc36,mfcc37,mfcc38,mfcc39,mfcc40,number
0,2022-12-08T11_08_28.068Z.wav,0.359336,0.112022,1857.209325,2283.210488,3857.673340,0.063302,-224.140121,117.112000,9.797302,...,3.142546,-5.397982,-2.406948,-3.560515,0.604840,-2.634303,0.067641,-4.910409,0.034435,1
1,2022-12-08T11_08_34.463Z.wav,0.369168,0.094388,1792.323699,2278.839639,3606.273193,0.064423,-259.522125,114.845924,14.378940,...,3.281986,-3.443275,-2.802702,-1.681797,-0.347466,-3.607420,-2.059134,-5.554461,-1.779652,1
2,2022-12-08T11_08_39.459Z.wav,0.387088,0.093372,1805.830107,2259.829943,3619.885254,0.065465,-275.060211,105.690338,19.432558,...,1.598398,-4.540739,-3.585951,-2.762686,-1.216613,-4.397162,-2.943735,-4.085824,-0.218240,1
3,2022-12-08T11_08_45.143Z.wav,0.355525,0.078548,1720.302523,2245.659829,3392.034472,0.064574,-276.694641,115.923302,22.807974,...,2.094329,-6.651269,-3.723580,-2.179221,-0.410426,-4.869701,-2.295571,-4.086039,0.290533,1
4,2022-12-08T11_08_50.643Z.wav,0.341803,0.064914,1900.735328,2379.611892,3921.236165,0.074006,-325.454102,107.094528,24.138638,...,1.682643,-2.467115,-1.667522,-1.783864,-0.007993,-1.257067,1.822308,-3.497823,-1.784715,1


In [7]:
feature  = ['chroma_stft','rmse', 'spectral_centroid', 
'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
'mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10',
'mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20',
'mfcc21','mfcc22','mfcc23','mfcc24','mfcc25','mfcc26','mfcc27','mfcc28','mfcc29','mfcc30',
'mfcc31','mfcc32','mfcc33','mfcc34','mfcc35','mfcc36','mfcc37','mfcc38','mfcc39','mfcc40']

X = data[feature]
Y = data.number

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [13]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)


clf = clf.fit(X_train,y_train)


y_pred = clf.predict(X_test)

In [14]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8205128205128205


In [15]:
import pickle
model_filename = 'SpeakUp.pkl'
pickle.dump(clf, open(model_filename,'wb'))